In [1]:
import numpy as np 
import pandas as pd

In [2]:
dataset = pd.read_csv('Financial Distress.csv')

In [3]:
X = dataset.iloc[:,:]
y = pd.DataFrame(X.iloc[:,2])
X.drop(columns = 'Financial Distress', inplace=True)

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
labelencoder_X_2 = LabelEncoder()
labelencoder_X_3 = LabelEncoder()

Companies = np.unique(X["Company"])
C = pd.DataFrame(labelencoder_X_1.fit_transform(X["Company"]))
onehotencoder = OneHotEncoder(categorical_features = [0])
C = pd.DataFrame(onehotencoder.fit_transform(C).toarray())
C.columns = Companies

Time = np.unique(X["Time"])
T = pd.DataFrame(labelencoder_X_2.fit_transform(X["Time"]))
onehotencoder = OneHotEncoder(categorical_features = [0])
T = pd.DataFrame(onehotencoder.fit_transform(T).toarray())
T.columns = Time

Features = np.unique(X["x80"])
F = pd.DataFrame(labelencoder_X_3.fit_transform(X["x80"]))
onehotencoder = OneHotEncoder(categorical_features = [0])
F = pd.DataFrame(onehotencoder.fit_transform(F).toarray())
F.columns = Features

P = pd.DataFrame(pd.concat((C,T),axis = 1))

X = X.drop(["Company","Time"],axis=1)
X = pd.DataFrame(pd.concat((P,X,F),axis = 1))

y = (y<-0.5)

C:\Users\darya\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
C:\Users\darya\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of in

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0,stratify = y)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score


class Metrics(Callback):
    def on_train_begin(self, logs={}):
     self.val_f1s = []
     self.val_recalls = []
     self.val_precisions = []
     
    def on_epoch_end(self, epoch, logs={}):  
     val_predict = self.model.predict_classes(self.validation_data[0])
     val_targ = self.validation_data[1]
     _val_f1 = f1_score(val_targ, val_predict)
     _val_recall = recall_score(val_targ, val_predict)
     _val_precision = precision_score(val_targ, val_predict)
     self.val_f1s.append(_val_f1)
     self.val_recalls.append(_val_recall)
     self.val_precisions.append(_val_precision)
     print (" — val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
     return
 
metrics = Metrics()

classifier = Sequential()

classifier.add(Dense(units = 30, kernel_initializer = "uniform", activation = 'relu', input_shape=(556,)))

# Adding the second hidden layer

classifier.add(Dense(units = 30, kernel_initializer = "uniform", activation = 'relu'))

# Adding the third hidden layer

classifier.add(Dense(units = 30, kernel_initializer = "uniform", activation = 'relu'))

# Adding the fourth hidden layer

classifier.add(Dense(units = 30, kernel_initializer = "uniform", activation = 'relu'))

# Adding the output layer

classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = 'sigmoid'))

# Compiling the ANN

classifier.compile(loss='binary_crossentropy',
          optimizer= "adam",
          metrics = ['accuracy']
          )




In [8]:
# Fitting the ANN to the Training set

classifier.fit(x=X_train,y=y_train,batch_size = 10, epochs = 100)


Epoch 1/100
257/257 [==============================] - 0s 2ms/step - loss: 0.2315 - accuracy: 0.9599
Epoch 2/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0897 - accuracy: 0.9630
Epoch 3/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0624 - accuracy: 0.9630
Epoch 4/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0453 - accuracy: 0.9669
Epoch 5/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0337 - accuracy: 0.9899
Epoch 6/100
257/257 [==============================] - 0s 1ms/step - loss: 0.0286 - accuracy: 0.9926
Epoch 7/100
257/257 [==============================] - 0s 1ms/step - loss: 0.0250 - accuracy: 0.9934
Epoch 8/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 0.9949
Epoch 9/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 0.9953
Epoch 10/100
257/257 [==============================] - 0s 2ms/step - loss: 0.0157 - accura

In [12]:
from sklearn.utils import class_weight
class_weight_real = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train['Financial Distress'])
                                               ,y_train['Financial Distress'])

classifier.fit(X_train, y_train, 
 validation_data=(X_test, y_test),
 epochs=200,
 batch_size=200)


y_pred = classifier.predict_classes(X_test)
y_pred2 = classifier.predict_classes(X_train)


cm = confusion_matrix(y_test , y_pred)

cm2 = confusion_matrix(y_train, y_pred2)

Epoch 1/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3410 - val_accuracy: 0.9564
Epoch 2/200
13/13 [==============================] - 0s 9ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3412 - val_accuracy: 0.9564
Epoch 3/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3414 - val_accuracy: 0.9564
Epoch 4/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3417 - val_accuracy: 0.9564
Epoch 5/200
13/13 [==============================] - 0s 12ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3419 - val_accuracy: 0.9564
Epoch 6/200
13/13 [==============================] - 0s 11ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3421 - val_accuracy: 0.9564
Epoch 7/200
13/13 [==============================] - 0s 7ms/step - loss: 0.0015 - accuracy: 0.9996 - val_loss: 0.3423 - val_accuracy: 0.9564
Epoch 8/20